In [ ]:
import csv
import concurrent.futures
from datetime import datetime

import requests

def get_movie_details(movie_id, api_key):
    """Fetch detailed movie information including budget and revenue."""
    url = f"https://api.themoviedb.org/3/movie/{movie_id}"
    params = {"api_key": api_key}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def get_movie_keywords(movie_id, api_key):
    """Fetch keywords for a movie."""
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/keywords"
    params = {"api_key": api_key}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return [keyword['name'] for keyword in response.json().get('keywords', [])]
    else:
        return []

def get_movie_cast(movie_id, api_key):
    """Fetch cast (actors) for a movie."""
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits"
    params = {"api_key": api_key}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return [cast['name'] for cast in response.json().get('cast', [])]
    else:
        return []

def get_movie_data(movie_id, api_key):
    """Fetch movie details, keywords, and cast concurrently."""
    with concurrent.futures.ThreadPoolExecutor() as executor:
        detail_future = executor.submit(get_movie_details, movie_id, api_key)
        keywords_future = executor.submit(get_movie_keywords, movie_id, api_key)
        cast_future = executor.submit(get_movie_cast, movie_id, api_key)

    detail_data = detail_future.result()
    keywords_data = keywords_future.result()
    cast_data = cast_future.result()

    if detail_data:
        detail_data['keywords'] = keywords_data
        detail_data['cast'] = cast_data

    return detail_data

def get_movies(api_key, start_date, end_date):
    """Fetch movies released within a specified date range from TMDB."""
    movies = []
    url = "https://api.themoviedb.org/3/discover/movie"
    page = 1
    total_pages = 1

    while page <= total_pages:
        params = {
            "api_key": api_key,
            "primary_release_date.gte": start_date,
            "primary_release_date.lte": end_date,
            "page": page
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            with concurrent.futures.ThreadPoolExecutor() as executor:
                future_to_movie_id = {executor.submit(get_movie_data, movie['id'], api_key): movie for movie in data['results']}
                for future in concurrent.futures.as_completed(future_to_movie_id):
                    movie_id = future_to_movie_id[future]['id']
                    movie_data = future.result()
                    if movie_data:
                        movies.append(movie_data)
            total_pages = data['total_pages']
            page += 1
        else:
            print(f"Error: Unable to fetch data for page {page}")
            break

    return movies

def save_movies_to_csv(movies, filename):
    """Save a list of movies to a CSV file."""
    if not movies:
        return

    keys = movies[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        for movie in movies:
            writer.writerow(movie)

# Your TMDB API key
api_key = "8289b64cd4ddd4f75deaad368ea6687b"

end_date = "2023-06-01"
start_date = '2023-01-01'

movies = get_movies(api_key, start_date, end_date)
print(f"Total movies found: {len(movies)}")

# Save movies to CSV
csv_filename = 'movies1.5.csv'
save_movies_to_csv(movies, csv_filename)
print(f"Movies saved to {csv_filename}")


C:\Users\anson\AppData\Local\Temp\ipykernel_50440\3019598660.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  movies = pd.read_csv('movies.csv', error_bad_lines=False)


,id,popularity,budget,revenue,runtime,vote_average,vote_count
count,722626.000000,722626.000000,7.226260e+05,7.226260e+05,6.882160e+05,722626.000000,722626.000000
mean,526917.266598,1.832502,3.970015e+05,9.878519e+05,1.989867e+02,2.444428,27.798878
std,253179.082009,20.710587,8.186191e+06,2.026544e+07,8.582798e+04,3.175982,384.041557
min,2.000000,0.600000,0.000000e+00,-1.200000e+01,0.000000e+00,0.000000,0.000000
25%,328928.000000,0.600000,0.000000e+00,0.000000e+00,5.000000e+00,0.000000,0.000000
50%,533361.500000,0.600000,0.000000e+00,0.000000e+00,5.000000e+01,0.000000,0.000000
75%,737830.000000,1.296000,0.000000e+00,0.000000e+00,9.000000e+01,5.600000,2.000000
max,968161.000000,8763.998000,5.000000e+09,2.920357e+09,5.050505e+07,10.000000,33262.000000
